In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv(r"C:\Users\LENOVO\Desktop\seeds.csv")

In [3]:
data.head()

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
data.shape

(199, 8)

In [5]:
f_v=data.iloc[:,0:7]

In [6]:
c_l=data[["Type"]]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test,y_train,y_test=train_test_split(f_v,c_l,test_size=0.2)

In [9]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
x_trains=std.fit_transform(x_train)

In [10]:
import keras.utils

In [11]:
y_trainf=keras.utils.to_categorical(y_train,dtype='int')[:,1:]

In [12]:
from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

In [13]:
import keras_tuner # pip install keras-tuner

In [14]:
model=Sequential()
model.add(Dense(10,activation="relu",input_dim=7))
model.add(Dense(7,activation="relu"))
model.add(Dense(3,activation="softmax"))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                80        
                                                                 
 dense_3 (Dense)             (None, 7)                 77        
                                                                 
 dense_4 (Dense)             (None, 3)                 24        
                                                                 
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [30]:
model.fit(x_trains,y_trainf,batch_size=10,epochs=5,validation_split=0.2)

Epoch 1/5
13/13 [==============================] - 4s 130ms/step - loss: 0.9154 - accuracy: 0.3858 - val_loss: 0.7481 - val_accuracy: 0.5625
Epoch 2/5
13/13 [==============================] - 0s 22ms/step - loss: 0.8718 - accuracy: 0.5039 - val_loss: 0.7215 - val_accuracy: 0.6250
Epoch 3/5
13/13 [==============================] - 0s 19ms/step - loss: 0.8411 - accuracy: 0.5827 - val_loss: 0.7019 - val_accuracy: 0.6250
Epoch 4/5
13/13 [==============================] - 1s 63ms/step - loss: 0.8162 - accuracy: 0.5984 - val_loss: 0.6855 - val_accuracy: 0.6250
Epoch 5/5
13/13 [==============================] - 0s 37ms/step - loss: 0.7961 - accuracy: 0.5906 - val_loss: 0.6732 - val_accuracy: 0.6250


In [65]:
def models(ob1):
    model=Sequential()
    
    unit=ob1.Int("neurons",1,30)
    unit2=ob1.Int("neurons2",1,30)
    model.add(Dense(unit,activation=ob1.Choice("act1",values=["relu","sigmoid","tanh"]),input_dim=7))
    model.add(Dropout(rate=ob1.Choice("dr_rate1",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(unit2,activation=ob1.Choice("act2",values=["relu","sigmoid","tanh"])))
    model.add(Dropout(rate=ob1.Choice("dr_rate2",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(3,activation="softmax"))
    
    model.compile(optimizer=ob1.Choice("opt",values=["rmsprop","adam","adagrad","sgd","gd"]),loss="categorical_crossentropy",metrics=["accuracy"])
    
    return model
    

In [68]:
kt=keras_tuner.RandomSearch(models,objective="val_accuracy",max_trials=4,directory=r"C:\Users\LENOVO\Desktop\deeplearning7")

In [69]:
kt.search(x_trains,y_trainf,epochs=6,validation_data=(x_trains,y_trainf))

Trial 4 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.4779874086380005
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit


In [70]:
kt.get_best_hyperparameters()[0].values

{'neurons': 21,
 'neurons2': 10,
 'act1': 'tanh',
 'dr_rate1': 0.7,
 'act2': 'tanh',
 'dr_rate2': 0.7,
 'opt': 'adagrad'}

In [71]:
modelf=kt.get_best_models(num_models=1)[0]

In [72]:
modelf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21)                168       
                                                                 
 dropout (Dropout)           (None, 21)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                220       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [73]:
modelf.fit(x_trains,y_trainf,batch_size=20,epochs=20,initial_epoch=6,validation_split=0.2)

Epoch 7/20
7/7 [==============================] - 2s 118ms/step - loss: 1.3523 - accuracy: 0.3858 - val_loss: 0.9382 - val_accuracy: 0.5625
Epoch 8/20
7/7 [==============================] - 0s 34ms/step - loss: 1.3199 - accuracy: 0.4724 - val_loss: 0.9326 - val_accuracy: 0.5625
Epoch 9/20
7/7 [==============================] - 0s 26ms/step - loss: 1.4000 - accuracy: 0.3622 - val_loss: 0.9262 - val_accuracy: 0.5625
Epoch 10/20
7/7 [==============================] - 0s 43ms/step - loss: 1.3808 - accuracy: 0.3780 - val_loss: 0.9199 - val_accuracy: 0.5625
Epoch 11/20
7/7 [==============================] - 1s 84ms/step - loss: 1.2370 - accuracy: 0.4488 - val_loss: 0.9130 - val_accuracy: 0.5625
Epoch 12/20
7/7 [==============================] - 0s 30ms/step - loss: 1.4633 - accuracy: 0.2913 - val_loss: 0.9081 - val_accuracy: 0.5625
Epoch 13/20
7/7 [==============================] - 1s 113ms/step - loss: 1.5169 - accuracy: 0.3071 - val_loss: 0.9024 - val_accuracy: 0.5625
Epoch 14/20
7/7 [====

In [101]:
def models(ob1):
    model=Sequential()
    x=0
    for i in range(ob1.Int("layers",2,10)):
        if x==0:
            model.add(Dense(ob1.Int("neuron"+str(x),1,10),activation="relu",input_dim=7))
        else:
            model.add(Dense(ob1.Int("neuron"+str(x),1,10),activation="relu"))
        x=x+1

    
    model.add(Dense(3,activation="softmax"))
    model.compile(optimizer=ob1.Choice("opt",values=["rmsprop","adam","adagrad","sgd","gd"]),loss="categorical_crossentropy",metrics=["accuracy"])
    
    return model

In [102]:
kt=keras_tuner.RandomSearch(models,objective="val_accuracy",max_trials=4,directory=r"C:\Users\LENOVO\Desktop\deeplearning12")

In [103]:
kt.search(x_trains,y_trainf,epochs=6,validation_data=(x_trains,y_trainf))

Trial 4 Complete [00h 00m 06s]
val_accuracy: 0.3836477994918823

Best val_accuracy So Far: 0.3836477994918823
Total elapsed time: 00h 00m 19s
INFO:tensorflow:Oracle triggered exit


In [104]:
kt.get_best_hyperparameters()[0].values

{'layers': 3, 'neuron0': 1, 'neuron1': 10, 'opt': 'adagrad', 'neuron2': 1}

In [87]:
modelf=kt.get_best_models(num_models=1)[0]

In [88]:
modelf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 48        
                                                                 
 dense_1 (Dense)             (None, 8)                 56        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
Total params: 347
Trainable params: 347
Non-trainable pa